# How to Implement a Quantitative Signal:

- This study accounts for common shortfalls and dire mistakes junior quants make when creating an Alpha

- These mistakes result in significant bias in backtesting, systemic risk in portfolio construction, & overall inaccurate performance results

- Principle of these mistakes is not accounting for the real-life delay in capturing a signal (i.e., you enter a position after, not simultaneously to the signal)

In [2]:
import pickle 
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt

from backtest_tools import portfolio_tools as pt, risk_analysis as ra

In [3]:
# Define investment universe of ETFs
assets = ['SPY']

# Returns of opening prices
open_asset_returns = pd.DataFrame()
# Returns of close prices
close_asset_returns = pd.DataFrame()

i = 0
for ticker in assets:
    prices = yf.download(ticker, start='1980-01-01', end=dt.date.today())
    
    # Get i'th asset's returns
    close_rets = prices['Adj Close'].pct_change().dropna()
    close_rets = close_rets.rename(ticker)
    open_rets = prices['Open'].pct_change().dropna()
    open_rets = open_rets.rename(ticker)

    close_asset_returns = pd.concat([close_asset_returns, close_rets], axis=1)
    open_asset_returns = pd.concat([open_asset_returns, open_rets], axis=1)

[*********************100%***********************]  1 of 1 completed


In [4]:
universe = ['SPY']

# Get Close Returns
returns = close_asset_returns[universe]
open_returns = open_asset_returns[universe]

Accurate Implementation of Strategy with Proper Offsetting - Accounts for Implementation Shortfall & Delay on Positioning

In [5]:
# Get dates where price is > 200 day moving average
long_dates = (pt.cumulative_returns(returns) > pt.cumulative_returns(returns).rolling(200).mean())

# Place binary "buy" position two days after signal... this captures the opening price returns for when the trade would be executed (two days after open price / one day after open price - 1)
buy_open_return_dates = long_dates.where((long_dates.SPY == True) & (long_dates.SPY.shift(1) == False)).shift(2) 

# Place binary "sell" signal two days after signal... longs will be closed the next moring, and there will be no position two days later
sell_open_return_dates = long_dates.where((long_dates.SPY == False) & (long_dates.SPY.shift(1) == True)).shift(2) 

# Strategy positions equate to shifting our long_dates up by two 
position_indicators = long_dates.SPY.shift(2)
position_indicators.iloc[np.where(position_indicators == True)] = 1
position_indicators.iloc[np.where(position_indicators == False)] = 0
position_indicators.dropna(inplace=True)

200 Day Moving Average Strategy Generates Long Signal as of 2023-03-21

In [6]:
position_indicators

1993-02-03    0
1993-02-04    0
1993-02-05    0
1993-02-08    0
1993-02-09    0
             ..
2023-03-27    1
2023-03-28    1
2023-03-29    1
2023-03-30    1
2023-03-31    1
Name: SPY, Length: 7595, dtype: object